In [59]:
import requests
import pandas as pd
import re

In [20]:
main_profiles = ['practitioner', 'organization', 'practitionerrole', 'endpoint', 'location', 'organizationaffiliation']

In [61]:
data_elements = []
element_mapping = {
    'name': 'HumanName',
    'telecom': 'ContactPoint'
}
for profile in main_profiles:
    json_schema = requests.get(f'https://hl7.org/fhir/R4B/{profile}.profile.json').json()
    for element in json_schema['snapshot']['element'][1:]:
        path_components = element['path'].split('.')
        field = path_components[1]
        if field not in ['meta', 'implicitRules', 'language', 'text', 'contained', 'extension', 'modifierExtension']:
            element = [elem for elem in json_schema['snapshot']['element'][1:] if elem['path'].split('.')[1]==field][0]
            if element['max'] == '*':
                cardinality = 'many'
            else:
                cardinality = 'one'
            if len(path_components)>2:
                subfield = path_components[2]
                if subfield not in ['meta', 'implicitRules', 'language', 'text', 'contained', 'extension', 'modifierExtension']:
                    data_elements.append({'fhirResource': profile, 'entity': field, 'cardinality':cardinality, 'original_field': subfield})
            else:
                try:
                    subprofile = field
                    if field in element_mapping.keys():
                        subprofile = element_mapping[field]
                    field_data = requests.get(f'https://hl7.org/fhir/R4B/{subprofile}.profile.json').json()
                    for field2 in field_data['snapshot']['element'][1:]:
                        subfield = field2['path'].split('.')[1]
                        if subfield not in ['meta', 'implicitRules', 'language', 'text', 'contained', 'extension', 'modifierExtension']:
                            data_elements.append({'fhirResource': profile, 'entity': field, 'cardinality':cardinality, 'original_field': subfield})
                except requests.JSONDecodeError:
                    data_elements.append({'fhirResource': profile, 'original_field': field, 'cardinality':cardinality})
                    pass
    

In [64]:
data_element_df = pd.DataFrame(data_elements).drop_duplicates()
data_element_df['data_element'] = data_element_df['original_field'].apply(lambda x: re.sub(r'(?<!^)(?=[A-Z])', '_', x).lower())

In [ ]:
data_element_df.to_csv('fhir_data_elements.csv', index=False)